<a href="https://colab.research.google.com/github/yuja28/sw_Bootcamp/blob/main/bottle_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# 데이터셋 경로 설정
train_data_dir = '/content/drive/MyDrive/mvtecv/bottle/train'
test_data_dir = '/content/drive/MyDrive/mvtecv/bottle/test'

# 이미지 리사이징을 위한 크기 설정
image_height, image_width = 224, 224


# 데이터셋 전처리를 위한 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(rescale=1./255)  # 훈련 데이터 정규화
test_datagen = ImageDataGenerator(rescale=1./255)   # 테스트 데이터 정규화


# 훈련 데이터의 'good' 카테고리 경로 설정
train_good_data_dir = os.path.join(train_data_dir, 'good')

# 훈련 데이터의 'good' 카테고리 이미지 로드
train_good_images = [os.path.join(train_good_data_dir, img) for img in os.listdir(train_good_data_dir)]

# 훈련 데이터의 'good' 카테고리 레이블 지정
train_good_labels = np.zeros(len(train_good_images))  # 정상 병태의 레이블은 0으로 지정



In [3]:
# 테스트 데이터의 이상 카테고리 이미지 로드 (broken_small + contamination)
test_anomaly_data_dirs = [os.path.join(test_data_dir, 'broken_small'), os.path.join(test_data_dir, 'contamination')]
test_anomaly_images = []
for anomaly_data_dir in test_anomaly_data_dirs:
    anomaly_images = [os.path.join(anomaly_data_dir, img) for img in os.listdir(anomaly_data_dir)]
    test_anomaly_images.extend(anomaly_images)

# 테스트 데이터의 이상 카테고리 레이블 지정 (이상은 1로 지정)
test_anomaly_labels = np.ones(len(test_anomaly_images))

# 테스트 데이터의 'good' 카테고리 경로 설정
test_good_data_dir = os.path.join(test_data_dir, 'good')

# 테스트 데이터의 'good' 카테고리 이미지 로드
test_good_images = [os.path.join(test_good_data_dir, img) for img in os.listdir(test_good_data_dir)]

# 테스트 데이터의 'good' 카테고리 레이블 지정
test_good_labels = np.zeros(len(test_good_images))  # 정상 병태의 레이블은 0으로 지정

In [4]:
# 훈련 데이터와 테스트 데이터의 이상 카테고리 이미지 및 레이블을 합쳐서 최종 데이터 생성
train_images = train_good_images
train_labels = train_good_labels

test_images = test_good_images + test_anomaly_images
test_labels = np.concatenate([test_good_labels, test_anomaly_labels])

# 데이터를 배열로 변환
train_images = np.array(train_images)
test_images = np.array(test_images)

# 데이터 전처리를 위해 데이터셋 생성
#학습데이터
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_height, image_width),
    batch_size=32,
    class_mode='binary'
)
#검증데이터
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(image_height, image_width),
    batch_size=32,
    class_mode='binary'
)

Found 209 images belonging to 1 classes.
Found 83 images belonging to 4 classes.


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

# 학습용 데이터와 검증용 데이터로 분리
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)
#CNN 모델 정의
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 구조 요약 출력
model.summary()

# 모델 학습
history = model.fit(
    train_generator,
    batch_size = 32,
    epochs = 10,
    validation_data=test_generator
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0